In [ ]:
from scripts.Model import Model
import torch
from transformers import RobertaForTokenClassification, RobertaTokenizerFast, TrainingArguments
from scripts.Reader import obtain_dataset, obtain_label_list

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Switching to GPU...")
    device = torch.device('cuda')
print("Current device index:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

In [ ]:
# Load tokenizer and model
label_list, label2id, id2label = obtain_label_list("TempEval3")
model_name = 'roberta-large'
tokenizer = RobertaTokenizerFast.from_pretrained(model_name, add_prefix_space=True)
model = RobertaForTokenClassification.from_pretrained(model_name, num_labels=len(label_list), label2id=label2id, id2label=id2label)

In [ ]:
model_instance = Model(
    device=device,
    model_name=model_name,
    tokenizer=tokenizer,
    model=model,
    label_list=label_list
)

tokenized_datasets = model_instance.tokenize_datasets(obtain_dataset("TempEval3", ["silver-1", "platinum"]))

In [ ]:
training_args = TrainingArguments(
    output_dir="./results/Timex3-NER",
    logging_dir="./logs/Timex3-NER",
    eval_strategy="steps",
    save_strategy="steps",
    logging_steps=100,
    num_train_epochs=3,
    save_total_limit=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

model_instance.set_training_args(training_args)

In [ ]:
model_instance.train(tokenized_datasets)

In [ ]:
model_instance.evaluate(tokenized_datasets)

In [ ]:
model_instance.print_classification_report(tokenized_datasets["test"]["ner_tags"], model_instance.predict(tokenized_datasets["test"]), id2label)

In [ ]:
tokenized_datasets